In [8]:
import splitfolders
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




In [9]:

# Split the dataset into train, val, and test (only run once)
splitfolders.ratio('PlantVillage', output='data_split', seed=42, ratio=(0.7, 0.15, 0.15))

Copying files: 20639 files [00:06, 3119.19 files/s]


In [10]:


# Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
    
)
# Only rescaling for validation and test
val_test_datagen = ImageDataGenerator(rescale=1. / 255)

# Load image data
train_gen = train_datagen.flow_from_directory(
    'data_split/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_test_datagen.flow_from_directory(
    'data_split/val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = val_test_datagen.flow_from_directory(
    'data_split/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)



Found 14440 images belonging to 15 classes.
Found 3089 images belonging to 15 classes.
Found 3109 images belonging to 15 classes.


In [11]:

# Load base model (without top layer)
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model initially

# Custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=output)


In [12]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('crop_disease_model.h5', save_best_only=True)
]

# Initial training (top layers only)
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks
)


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2780 - loss: 2.3500

452/452 ━━━━━━━━━━━━━━━━━━━━ 632s 1s/step - accuracy: 0.2783 - loss: 2.3490 - val_accuracy: 0.6837 - val_loss: 1.0878
Epoch 2/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 803ms/step - accuracy: 0.6100 - loss: 1.2071

452/452 ━━━━━━━━━━━━━━━━━━━━ 409s 905ms/step - accuracy: 0.6101 - loss: 1.2070 - val_accuracy: 0.7488 - val_loss: 0.8040
Epoch 3/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 725ms/step - accuracy: 0.6901 - loss: 0.9539

452/452 ━━━━━━━━━━━━━━━━━━━━ 373s 824ms/step - accuracy: 0.6902 - loss: 0.9538 - val_accuracy: 0.7828 - val_loss: 0.6889
Epoch 4/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 726ms/step - accuracy: 0.7219 - loss: 0.8408

...

452/452 ━━━━━━━━━━━━━━━━━━━━ 374s 827ms/step - accuracy: 0.8256 - loss: 0.5147 - val_accuracy: 0.8637 - val_loss: 0.4215
Epoch 16/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 728ms/step - accuracy: 0.8307 - loss: 0.5027

452/452 ━━━━━━━━━━━━━━━━━━━━ 374s 827ms/step - accuracy: 0.8307 - loss: 0.5027 - val_accuracy: 0.8653 - val_loss: 0.4121
Epoch 17/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 725ms/step - accuracy: 0.8284 - loss: 0.5081

452/452 ━━━━━━━━━━━━━━━━━━━━ 373s 825ms/step - accuracy: 0.8284 - loss: 0.5081 - val_accuracy: 0.8647 - val_loss: 0.4074
Epoch 18/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 732ms/step - accuracy: 0.8396 - loss: 0.4879

452/452 ━━━━━━━━━━━━━━━━━━━━ 376s 832ms/step - accuracy: 0.8395 - loss: 0.4879 - val_accuracy: 0.8695 - val_loss: 0.3965
Epoch 19/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 728ms/step - accuracy: 0.8430 - loss: 0.4708

452/452 ━━━━━━━━━━━━━━━━━━━━ 374s 827ms/step - accuracy: 0.8430 - loss: 0.4708 - val_accuracy: 0.8679 - val_loss: 0.3951
Epoch 20/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 375s 829ms/step - accuracy: 0.8407 - loss: 0.4645 - val_accuracy: 0.8705 - val_loss: 0.3958


In [13]:
# Fine-tune base model
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tuning training
model.fit(train_gen, validation_data=val_gen, epochs=5)

# Evaluate the model
loss, acc = model.evaluate(test_gen)
print(f" Test Accuracy: {acc * 100:.2f}%")


Epoch 1/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 514s 1s/step - accuracy: 0.5963 - loss: 1.3652 - val_accuracy: 0.8521 - val_loss: 0.4412
Epoch 2/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 511s 1s/step - accuracy: 0.8057 - loss: 0.5781 - val_accuracy: 0.8569 - val_loss: 0.4337
Epoch 3/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 510s 1s/step - accuracy: 0.8501 - loss: 0.4533 - val_accuracy: 0.8773 - val_loss: 0.3658
Epoch 4/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 512s 1s/step - accuracy: 0.8637 - loss: 0.4192 - val_accuracy: 0.8896 - val_loss: 0.3170
Epoch 5/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 513s 1s/step - accuracy: 0.8855 - loss: 0.3393 - val_accuracy: 0.8971 - val_loss: 0.2819
98/98 ━━━━━━━━━━━━━━━━━━━━ 66s 672ms/step - accuracy: 0.9079 - loss: 0.2567
✅ Test Accuracy: 90.83%


In [14]:
# Save the trained model for use in Streamlit or Gradio app
model.save('final_model.h5')
